In [3]:
import torch
from enformer_pytorch import from_pretrained
from torchinfo import summary

In [26]:
enformer = from_pretrained(
    "EleutherAI/enformer-official-rough", use_tf_gamma=False
).cuda()
summary(enformer)

Layer (type:depth-idx)                             Param #
Enformer                                           --
├─Sequential: 1-1                                  --
│    └─Conv1d: 2-1                                 46,848
│    └─Residual: 2-2                               --
│    │    └─Sequential: 3-1                        592,128
│    └─AttentionPool: 2-3                          --
│    │    └─Rearrange: 3-2                         --
│    │    └─Conv2d: 3-3                            589,824
├─Sequential: 1-2                                  --
│    └─Sequential: 2-4                             --
│    │    └─Sequential: 3-4                        2,951,424
│    │    └─Residual: 3-5                          592,128
│    │    └─AttentionPool: 3-6                     589,824
│    └─Sequential: 2-5                             --
│    │    └─Sequential: 3-7                        3,443,072
│    │    └─Residual: 3-8                          805,504
│    │    └─AttentionPool: 3-9    

In [27]:
data = torch.load("../data/test-sample.pt")

In [9]:
for k, v in data.items():
    print(k, v.shape, v.dtype)

sequence torch.Size([131072, 4]) torch.float32
target torch.Size([896, 5313]) torch.float32


In [10]:
import numpy as np

In [15]:
sequence = np.load("test_sequence.npy")
sequence = torch.from_numpy(sequence).to("cuda")

In [22]:
with torch.no_grad():
    result = enformer(sequence)

In [28]:
seq, target = data["sequence"].cuda(), data["target"].cuda()

with torch.no_grad():
    corr_coef = enformer(seq, target=target, return_corr_coef=False, head="human")

print(corr_coef)
assert corr_coef > 0.1

tensor(0.6000, device='cuda:0')
